In [ ]:
# |default_exp transcription
# |default_cls_lvl 3

In [ ]:
# |hide
%reload_ext autoreload
%autoreload 2

# transcription

Classes and methods for transcribing video/audio into text

In [ ]:
# |export
from __future__ import annotations

from course_copilot import utils

In [ ]:
# | hide
import pdb

from fastcore.test import *
import nbdev

from blurr.utils import print_versions

## Setup

## Export -

In [ ]:
# | hide
nbdev.nbdev_export()